In [33]:
import requests                         # 웹 문서를 text형태로 가져오기 위해 필요
from bs4 import BeautifulSoup as bs     # 가져온 text형태의 문서의 태그에 접근할 수 있게 정제
import pandas as pd                     # 데이터 분석
from selenium import webdriver          # 웹 브라우저 컨트롤 (막힌 사이트 스크래핑 용)
from urllib.request import urlretrieve  # 파일 다운로드 관련
import re                               # 정규식 사용
import time
from selenium.webdriver.common.by import By     # 브라우저 조작 관련
from selenium.webdriver.common.keys import Keys # 브라우저 key 입력 관련

from pykrx import stock                 # 주가 조회 모듈
import yfinance as yf                   # 해외 주가 조회 모듈
import pybithumb as pbit                # 빗썸 API

import folium                           # 데이터 분석

In [30]:
'''
스타벅스
'''

driver = webdriver.Chrome('chromedriver.exe')                   # 크롬을 이용해 불러옴
driver.get('https://www.starbucks.co.kr/store/store_map.do')

time.sleep(2)                                                   # 페이지 로드 전에 하위 코드 실행 방지

area_btn = driver.find_element(By.CSS_SELECTOR, '.loca_search h3 a')
area_btn.click()
time.sleep(1)

seoul_btn = driver.find_element(By.CSS_SELECTOR, '.set_sido_cd_btn')
seoul_btn.click()
time.sleep(1)

all_btn = driver.find_element(By.CSS_SELECTOR, '.set_gugun_cd_btn')
all_btn.click()
time.sleep(2)

html = driver.page_source
bsObj = bs(html)

li = bsObj.select('.quickSearchResultBoxSidoGugun .quickResultLstCon')
ap = bsObj.select('.quickSearchResultBoxSidoGugun .quickResultLstCon > .result_details')
stype = bsObj.select('.quickSearchResultBoxSidoGugun .quickResultLstCon > i')

print(len(li))
print(len(stype))



storeData = []
for i in range(len(li)):
    apData = str(ap[i]).split('<br/>')
    storeData.append([
        li[i]['data-lat'],
        li[i]['data-long'],
        li[i]['data-name'],
        bs(apData[0]).text,
        bs(apData[1]).text,
        stype[i].text
        ])
df = pd.DataFrame(storeData, columns=['위도', '경도', '매장명', '주소', '전화번호', '매장 종류'])
df

C:\Users\BeomSeok\AppData\Local\Temp\ipykernel_102244\248129128.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')                   # 크롬을 이용해 불러옴


600
600


,위도,경도,매장명,주소,전화번호,매장 종류
0,37.501087,127.043069,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),1522-3232,리저브 매장 2번
1,37.510178,127.022223,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),1522-3232,리저브 매장 2번
2,37.5139309,127.0206057,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),1522-3232,리저브 매장 2번
3,37.499517,127.031495,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,리저브 매장 2번
4,37.494668,127.062583,대치재경빌딩R,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,리저브 매장 2번
...,...,...,...,...,...,...
595,37.579594,127.087966,사가정역,서울특별시 중랑구 면목로 310,1522-3232,리저브 매장 2번
596,37.59689,127.08647,상봉역,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,리저브 매장 2번
597,37.615368,127.076633,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,리저브 매장 2번
598,37.6066536267232,127.106359790053,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,리저브 매장 2번


In [31]:
# 엑셀로 저장
df.to_excel('starbucks_seoul.xlsx', index=False)
# 엑셀 가져오기
# df.read_excel('starbucks_seoul.xlsx', header=False)


In [58]:
map = folium.Map(
    location=[37.5666805, 126.9784147],      # 중심 위도, 경도
    zoom_start=13,                           # zoom 설정 (낮을수록 많이 보임)
    tiles='Stamen Terrain'                   # 지도 모양 설정
)

# 마커 찍기
folium.CircleMarker(
    location=[37.5666805, 126.9784147],
    fill=True
).add_to(map)

# 스타벅스 마커 찍기
for store in storeData:
    folium.Marker(
        location=[store[0], store[1]],
        tooltip=store[2],
        popup=store[3],
    ).add_to(map)
map